# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [3]:
import pandas as pd

### Загрузка данных и предварительный анализа

In [4]:
data = pd.read_csv("lastfm.csv")
data.sample(10)

,user,artist,sex,country
245473,16701,jimi hendrix,m,Germany
38160,2599,camel,m,United States
212018,14415,arctic monkeys,m,Latvia
102220,6963,eagles,m,United Kingdom
183876,12528,low,m,Ireland
227029,15424,beastie boys,m,United States
216424,14704,joy division,m,Iceland
85539,5826,bob marley,f,United States
198419,13486,faithless,m,Netherlands
51492,3501,chamillionaire,m,Russian Federation


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.8+ MB


In [6]:
data['artist']

0           red hot chili peppers
1         the black dahlia murder
2                       goldfrapp
3                dropkick murphys
4                        le tigre
                   ...           
289950                  bob dylan
289951                     pixies
289952                  the clash
289953       a tribe called quest
289954                  radiohead
Name: artist, Length: 289955, dtype: object

### Выбор данных по странам: Belarus, Ukraine, Poland, Russian Federation (3530203_70102)

In [7]:
data.loc[data['country'].isin(['Belarus', 'Ukraine', 'Poland', 'Russian Federation'])]

,user,artist,sex,country
490,35,radiohead,m,Ukraine
491,35,the kooks,m,Ukraine
492,35,coldplay,m,Ukraine
493,35,nine inch nails,m,Ukraine
494,35,hot chip,m,Ukraine
...,...,...,...,...
289877,19713,linkin park,m,Ukraine
289878,19713,avril lavigne,m,Ukraine
289879,19713,the prodigy,m,Ukraine
289880,19713,blank & jones,m,Ukraine


In [8]:
df = pd.DataFrame(data.loc[data['country'].isin(['Belarus', 'Ukraine', 'Poland', 'Russian Federation'])])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32376 entries, 490 to 289881
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     32376 non-null  int64 
 1   artist   32376 non-null  object
 2   sex      32376 non-null  object
 3   country  32376 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [10]:
df.groupby('user').size()

user
35       33
39       11
50       30
64       12
82       24
         ..
19703    44
19706     4
19710     8
19712     9
19713    12
Length: 2007, dtype: int64

In [11]:
df.groupby('user').size().describe()

count    2007.000000
mean       16.131540
std         9.629354
min         1.000000
25%         8.000000
50%        15.000000
75%        23.000000
max        48.000000
dtype: float64

In [12]:
df.sample(5)

,user,artist,sex,country
227097,15428,alesana,f,Russian Federation
21491,1423,tristania,f,Poland
287151,19537,fatboy slim,m,Poland
259251,17596,tom waits,m,Russian Federation
7775,522,tori amos,f,Poland


УБИРАЕМ ПРОПУСКИ

In [13]:
df['artist'].unique()

array(['radiohead', 'the kooks', 'coldplay', 'nine inch nails',
       'hot chip', 'black rebel motorcycle club', 'lcd soundsystem',
       'kasabian', 'n*e*r*d', 'editors', 'klaxons', 'bloc party',
       'the cinematic orchestra', 'depeche mode', 'girl talk', 'lamb',
       'cut copy', '2pac', 'iamx', 'the raconteurs',
       'antony and the johnsons', 'trentemøller', 'unkle', 'the presets',
       'the killers', 'gorillaz', 'moloko', 'massive attack', 'mgmt',
       'portishead', 'the cardigans', 'kylie minogue', 'patrick wolf',
       'britney spears', 'a-ha', 'joss stone', 'christina aguilera',
       'hilary duff', 'katie melua', 'katy perry', 'norah jones',
       'september', 'shakira', 'leona lewis', 'the ataris',
       'yann tiersen', 'the smashing pumpkins', 'carla bruni', 'the cure',
       'joy division', 'placebo', 'blink-182', '65daysofstatic', 'beirut',
       'nirvana', 'blonde redhead', 'underoath', 'regina spektor',
       'yeah yeah yeahs', 'bauhaus', 'mew', 'takin

#### Пропущенные значения обозначены как 'unknown'

In [14]:
df['artist'].loc[df['artist'].str.contains('\.')].head(10)

1215        p.o.d.
1943      o.s.t.r.
2736        r.e.m.
2751        r.e.m.
2822        r.e.m.
3821        r.e.m.
4183        r.e.m.
4207        r.e.m.
4405        t. rex
5700    mr. scruff
Name: artist, dtype: object

In [15]:
df['artist'].loc[df['artist'].str.contains(',')]

Series([], Name: artist, dtype: object)

##### в названиях групп не встречается ',' поэтому используем как разделитель

##### также среди значений artist встречается '[unknown]' - это пропуски от которых нужно избавиться

In [17]:
df.loc[(df.artist=='[unknown]')]

,user,artist,sex,country
2651,185,[unknown],f,Poland
3814,260,[unknown],m,Poland
15807,1043,[unknown],f,Poland
17624,1158,[unknown],f,Russian Federation
20016,1316,[unknown],f,Poland
...,...,...,...,...
277855,18889,[unknown],m,Russian Federation
281579,19155,[unknown],m,Russian Federation
282940,19256,[unknown],m,Poland
284124,19339,[unknown],m,Ukraine


103 строки со значением unknown

In [19]:
df = df.drop(df[(df.artist=='[unknown]')].index)
df

,user,artist,sex,country
490,35,radiohead,m,Ukraine
491,35,the kooks,m,Ukraine
492,35,coldplay,m,Ukraine
493,35,nine inch nails,m,Ukraine
494,35,hot chip,m,Ukraine
...,...,...,...,...
289877,19713,linkin park,m,Ukraine
289878,19713,avril lavigne,m,Ukraine
289879,19713,the prodigy,m,Ukraine
289880,19713,blank & jones,m,Ukraine


в исходном 32376 - 103 = 32273 rows без пропусков

In [20]:
pd.set_option('display.max_colwidth', None)
df_by_user = df.groupby(['country','user', 'sex'])['artist'].apply(lambda x: ",".join(x)).reset_index()
df_by_user

,country,user,sex,artist
0,Belarus,387,m,"depeche mode,mylo,u2,mgmt,pulp,air"
1,Belarus,1167,m,"rammstein,dethklok,therion,behemoth,lacrimosa,john williams,the 69 eyes,the birthday massacre,amon amarth,deathstars,combichrist,him,samael,nightwish"
2,Belarus,1967,f,"the beatles,rammstein,billy talent,my chemical romance,good charlotte,oomph!,muse,evanescence,metallica,system of a down,bullet for my valentine,linkin park,the rasmus,franz ferdinand,scorpions"
3,Belarus,2303,m,"dark tranquillity,scorpions,bullet for my valentine,nightwish,opeth"
4,Belarus,2326,m,"flyleaf,akira yamaoka,stratovarius,joe satriani,the white stripes,bloodhound gang,metallica,gogol bordello,alice cooper,pink floyd,manowar,soundtrack,anathema,louis armstrong,the offspring"
...,...,...,...,...
2002,Ukraine,19339,m,"cypress hill,atb,limp bizkit,pendulum,moby,the prodigy,blank & jones,dr. dre,bob marley & the wailers,armin van buuren,groove armada,notorious b.i.g.,asian dub foundation,crystal castles,gorillaz,oasis"
2003,Ukraine,19355,m,"jedi mind tricks,immortal technique"
2004,Ukraine,19428,m,"arctic monkeys,keane,travis,lostprophets,the killers,papa roach,the fall of troy,mudvayne,placebo,30 seconds to mars,emery,paramore,as i lay dying,blur,the rapture,lifehouse"
2005,Ukraine,19572,m,"portishead,the killers,m83,armin van buuren,interpol,caliban,as i lay dying,bloc party,hatebreed,joy division,röyksopp,tiësto,atb,coldplay,the kooks,heaven shall burn,cocorosie,arctic monkeys,kosheen,sigur rós,the chemical brothers,radiohead"


In [22]:
dummy_df_by_user = df_by_user['artist'].str.get_dummies(',')

In [24]:
df_by_user['artist'].str.get_dummies(',')

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,...,weezer,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,zero 7,Édith piaf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2004,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
pd.set_option('display.max_column', None)
df_by_user_ext = pd.concat([df_by_user,dummy_df_by_user],axis=1)
df_by_user_ext.sample(5)

,country,user,sex,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,a fine frenzy,a perfect circle,a tribe called quest,a-ha,abba,above & beyond,ac/dc,adam green,adele,adema,aerosmith,aesop rock,afi,after forever,against me!,agalloch,aimee mann,air,akira yamaoka,akon,akurat,al green,alanis morissette,alesana,alexisonfire,alice cooper,alice in chains,alicia keys,alkaline trio,all shall perish,all that remains,all time low,alter bridge,amon amarth,amon tobin,amorphis,amy macdonald,amy winehouse,anathema,anberlin,andrew bird,andrew lloyd webber,angels & airwaves,angra,ani difranco,animal collective,anna ternheim,anthrax,anti-flag,antonio vivaldi,antony and the johnsons,aphex twin,apocalyptica,apoptygma berzerk,apparat,arcade fire,arch enemy,architecture in helsinki,archive,arctic monkeys,aretha franklin,armin van buuren,as i lay dying,ashlee simpson,asian dub foundation,at the drive-in,at the gates,atb,atmosphere,atreyu,audioslave,autechre,avantasia,avenged sevenfold,avril lavigne,ayreon,babyshambles,backstreet boys,bad brains,bad religion,badly drawn boy,band of horses,barenaked ladies,basement jaxx,basshunter,bat for lashes,bathory,battles,bauhaus,beastie boys,beatsteaks,beck,bee gees,behemoth,beirut,belle and sebastian,ben folds,ben folds five,ben harper,benny benassi,beyoncé,biffy clyro,billie holiday,billy joel,billy talent,björk,black eyed peas,black flag,black label society,black rebel motorcycle club,black sabbath,blackfield,blank & jones,blind guardian,blink-182,bloc party,blonde redhead,blondie,bloodbath,bloodhound gang,blur,boards of canada,bob dylan,bob marley,bob marley & the wailers,bon iver,bon jovi,bonnie prince billy,bonobo,booka shade,bowling for soup,boys like girls,boys noize,brand new,breaking benjamin,brian eno,bright eyes,bring me the horizon,britney spears,broken social scene,bruce springsteen,bryan adams,buckethead,buena vista social club,built to spill,bullet for my valentine,burial,burzum,bush,busta rhymes,buzzcocks,böhse onkelz,caetano veloso,cake,calexico,caliban,calvin harris,camel,camera obscura,cannibal corpse,cansei de ser sexy,captain beefheart & his magic band,carcass,carla bruni,cascada,cat power,cat stevens,chamillionaire,charlie parker,chet baker,chevelle,chicane,children of bodom,chimaira,chiodos,chris brown,chris cornell,christina aguilera,chromeo,circa survive,city and colour,clap your hands say yeah,clint mansell,clueso,cobra starship,cocorosie,cocteau twins,coheed and cambria,colbie caillat,cold war kids,coldplay,coma,combichrist,comeback kid,common,converge,corinne bailey rae,counting crows,cradle of filth,craig david,cream,creed,creedence clearwater revival,crystal castles,cunninlynguists,cursive,cut copy,cute is what we aim for,cypress hill,céline dion,daft punk,damian marley,damien rice,danny elfman,dark tranquillity,darkthrone,dashboard confessional,dave matthews band,david bowie,david gray,david guetta,de la soul,de-phazz,dead can dance,dead kennedys,death,death cab for cutie,death from above 1979,deathstars,deep purple,deerhoof,deerhunter,def leppard,deftones,delerium,depeche mode,descendents,dethklok,deus,devendra banhart,devildriver,devo,diana krall,dido,die apokalyptischen reiter,die toten hosen,die Ärzte,digitalism,dimmu borgir,dinosaur jr.,dio,dir en grey,dire straits,dirty pretty things,disturbed,dizzee rascal,dj krush,dj shadow,django reinhardt,dmx,dope,doves,down,dr. dre,dragonforce,dream theater,dredg,dropkick murphys,drowning pool,duffy,duran duran,eagles,eagles of death metal,echo & the bunnymen,eddie vedder,edguy,editors,eels,einstürzende neubauten,elbow,electric light orchestra,electric six,ella fitzgerald,elliott smith,elton john,eluveitie,elvis costello,elvis presley,emery,emilie autumn,emilie simon,emilíana torrini,eminem,enigma,ennio morricone,enrique iglesias,ensiferum,enter shikari,enya,epica,equilibrium,era,eric clapton,erykah badu,escape the fate,evanescence,ex

In [119]:
df_by_user_ext.columns

Index(['country', 'user', 'sex', 'artist',
       '...and you will know us by the trail of dead', '2pac', '3 doors down',
       '30 seconds to mars', '311', '36 crazyfists',
       ...
       'weezer', 'wilco', 'within temptation', 'wolfgang amadeus mozart',
       'wu-tang clan', 'yann tiersen', 'yeah yeah yeahs', 'yellowcard',
       'zero 7', 'Édith piaf'],
      dtype='object', length=997)

### Извлечение правил с помощью Mlxtend (machine learning extensions) 

Python library of useful tools for the day-to-day data science tasks.

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

#### APRIORI

In [36]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax
apr_itemsets = apriori(df_by_user_ext.drop(columns=['user','country','sex','artist']), 
                            min_support=0.01, use_colnames=True)

apr_itemsets.sort_values(by = ['support'], ascending=False)

apr_itemsets

,support,itemsets
0,0.031888,(2pac)
1,0.027902,(3 doors down)
2,0.059791,(30 seconds to mars)
3,0.024415,(50 cent)
4,0.021923,(65daysofstatic)
...,...,...
1305,0.011460,"(red hot chili peppers, placebo, muse)"
1306,0.011958,"(placebo, the killers, muse)"
1307,0.011460,"(red hot chili peppers, radiohead, muse)"
1308,0.010463,"(red hot chili peppers, nirvana, system of a down)"


In [31]:
rules = association_rules(apr_itemsets, metric="confidence", min_threshold=0.5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022
1,(above & beyond),(armin van buuren),0.021923,0.038864,0.014449,0.659091,16.958916,0.013597,2.819332
2,(above & beyond),(atb),0.021923,0.048829,0.011460,0.522727,10.705241,0.010389,1.992930
3,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879
4,(above & beyond),(ferry corsten),0.021923,0.022422,0.011958,0.545455,24.327273,0.011467,2.150673
...,...,...,...,...,...,...,...,...,...
67,"(placebo, nirvana)",(muse),0.022920,0.101644,0.012456,0.543478,5.346867,0.010127,1.967827
68,"(nirvana, muse)",(placebo),0.024415,0.102641,0.012456,0.510204,4.970775,0.009950,1.832108
69,"(red hot chili peppers, placebo)",(muse),0.020927,0.101644,0.011460,0.547619,5.387605,0.009333,1.985839
70,"(the killers, muse)",(placebo),0.022422,0.102641,0.011958,0.533333,5.196117,0.009657,1.922913


In [32]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022,1
1,(above & beyond),(armin van buuren),0.021923,0.038864,0.014449,0.659091,16.958916,0.013597,2.819332,1
2,(above & beyond),(atb),0.021923,0.048829,0.011460,0.522727,10.705241,0.010389,1.992930,1
3,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879,1
4,(above & beyond),(ferry corsten),0.021923,0.022422,0.011958,0.545455,24.327273,0.011467,2.150673,1
5,(above & beyond),(paul van dyk),0.021923,0.030892,0.012955,0.590909,19.128299,0.012277,2.368931,1
6,(blank & jones),(armin van buuren),0.021923,0.038864,0.010962,0.500000,12.865385,0.010110,1.922272,1
7,(ferry corsten),(armin van buuren),0.022422,0.038864,0.015446,0.688889,17.725641,0.014575,3.089366,1
8,(paul van dyk),(armin van buuren),0.030892,0.038864,0.016442,0.532258,13.695409,0.015242,2.054843,1
9,(tiësto),(armin van buuren),0.030892,0.038864,0.015446,0.500000,12.865385,0.014245,1.922272,1


#### Проверяем разброс значений lift, чтобы определить сильные правила

In [34]:
rules['lift'].describe()

count    72.000000
mean     10.960966
std       6.921340
min       3.874517
25%       5.077096
50%       9.852114
75%      13.503651
max      30.409091
Name: lift, dtype: float64

In [53]:
rules[(rules['lift']>9.8)&(rules['antecedents'].apply(lambda x: len(x)) >= 2)].sort_values('lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
31,"(armin van buuren, paul van dyk)",(above & beyond),0.016442,0.021923,0.010962,0.666667,30.409091,0.010601,2.934230
49,"(armin van buuren, atb)",(ferry corsten),0.017937,0.022422,0.010463,0.583333,26.016667,0.010061,2.346188
32,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
48,"(ferry corsten, atb)",(armin van buuren),0.011958,0.038864,0.010463,0.875000,22.514423,0.009999,7.689088
33,"(paul van dyk, above & beyond)",(armin van buuren),0.012955,0.038864,0.010962,0.846154,21.772189,0.010458,6.247384
47,"(ferry corsten, armin van buuren)",(atb),0.015446,0.048829,0.010463,0.677419,13.873272,0.009709,2.948630
40,"(red hot chili peppers, happysad)",(akurat),0.016442,0.048331,0.010463,0.636364,13.166823,0.009669,2.617090
37,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
38,"(pidżama porno, akurat)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
57,"(pidżama porno, myslovitz)",(happysad),0.015446,0.053812,0.010463,0.677419,12.588710,0.009632,2.933184


In [58]:
rules['support'].describe()

count    72.000000
mean      0.012595
std       0.003140
min       0.010463
25%       0.010962
50%       0.011709
75%       0.012955
max       0.031888
Name: support, dtype: float64

In [60]:
rules[ 
       (rules['lift'] > 10) & (rules['support'] > 0.0117) 
    & (rules['antecedents'].apply(lambda x: len(x)) >= 2)
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
38,"(pidżama porno, akurat)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
39,"(happysad, akurat)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182


#### по support

In [65]:
rules[(rules['antecedents'].apply(lambda x: len(x)) >= 2)].sort_values(by = ['support'], ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
38,"(pidżama porno, akurat)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
39,"(happysad, akurat)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182
59,"(koЯn, system of a down)",(linkin park),0.025909,0.107623,0.012955,0.500000,4.645833,0.010166,1.784753
60,"(koЯn, linkin park)",(system of a down),0.019930,0.106129,0.012955,0.650000,6.124648,0.010839,2.553918


In [42]:
rules = association_rules(apr_itemsets, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >=2) & (rules['confidence'] > 0.5) & (rules['lift'] > 5) & (rules['conviction'] > 1.2)].sort_values(by = ['support'], ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
38,"(pidżama porno, akurat)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
39,"(happysad, akurat)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182
60,"(koЯn, linkin park)",(system of a down),0.019930,0.106129,0.012955,0.650000,6.124648,0.010839,2.553918
67,"(placebo, nirvana)",(muse),0.022920,0.101644,0.012456,0.543478,5.346867,0.010127,1.967827
50,"(portishead, björk)",(massive attack),0.021923,0.080219,0.011958,0.545455,6.799548,0.010199,2.023518
70,"(the killers, muse)",(placebo),0.022422,0.102641,0.011958,0.533333,5.196117,0.009657,1.922913
51,"(massive attack, björk)",(portishead),0.018934,0.074738,0.011958,0.631579,8.450526,0.010543,2.511424
35,"(happysad, myslovitz)",(akurat),0.020927,0.048331,0.011460,0.547619,11.330633,0.010448,2.103690
36,"(myslovitz, akurat)",(happysad),0.018435,0.053812,0.011460,0.621622,11.551802,0.010468,2.500641


In [44]:
max_res=fpmax(df_by_user_ext.drop(columns=['user','country','sex','artist']),min_support=0.01, use_colnames=True)
max_res.sort_values(by = ['support'], ascending=False)

,support,itemsets
533,0.037867,(clint mansell)
534,0.037867,(tori amos)
509,0.035874,(boards of canada)
505,0.035874,(god is an astronaut)
495,0.035376,(burial)
...,...,...
592,0.010463,"(arctic monkeys, oasis)"
584,0.010463,"(rage against the machine, limp bizkit)"
953,0.010463,"(nirvana, queen)"
563,0.010463,"(amon tobin, the prodigy)"


In [45]:
rules = association_rules(apr_itemsets, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >=2) & 
      (rules['confidence'] > 0.75) & (rules['lift'] > 5) & 
      (rules['conviction'] > 4)
].sort_values(by = ['support'], ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
33,"(paul van dyk, above & beyond)",(armin van buuren),0.012955,0.038864,0.010962,0.846154,21.772189,0.010458,6.247384
48,"(ferry corsten, atb)",(armin van buuren),0.011958,0.038864,0.010463,0.875000,22.514423,0.009999,7.689088


#### FPGROWTH

In [29]:
fpgr_itemsets = fpgrowth(df_by_user_ext.drop(columns=['user','country','sex','artist']), 
                            min_support=0.01, use_colnames=True)

fpgr_itemsets.sort_values('support',ascending=False)


,support,itemsets
18,0.129048,(metallica)
48,0.123568,(the prodigy)
67,0.122073,(red hot chili peppers)
105,0.114599,(radiohead)
54,0.112108,(coldplay)
...,...,...
822,0.010463,"(the offspring, avril lavigne)"
819,0.010463,"(the killers, green day)"
796,0.010463,"(the prodigy, atb)"
1194,0.010463,"(pidżama porno, happysad, myslovitz)"


In [68]:
rules = association_rules(fpgr_itemsets, metric="confidence", min_threshold=0.5)
rules.sort_values('lift',ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,"(armin van buuren, paul van dyk)",(above & beyond),0.016442,0.021923,0.010962,0.666667,30.409091,0.010601,2.934230
46,(above & beyond),"(armin van buuren, paul van dyk)",0.021923,0.016442,0.010962,0.500000,30.409091,0.010601,1.967115
31,"(armin van buuren, atb)",(ferry corsten),0.017937,0.022422,0.010463,0.583333,26.016667,0.010061,2.346188
44,"(armin van buuren, above & beyond)",(paul van dyk),0.014449,0.030892,0.010962,0.758621,24.557286,0.010515,4.014877
41,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879


In [67]:

rules[ 
       (rules['lift'] > 0.7) &
       (rules['support'] > 0.016)
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(a perfect circle),(tool),0.039860,0.056303,0.022422,0.562500,9.990597,0.020177,2.157022
8,(paul van dyk),(armin van buuren),0.030892,0.038864,0.016442,0.532258,13.695409,0.015242,2.054843
12,(breaking benjamin),(linkin park),0.035874,0.107623,0.018435,0.513889,4.774884,0.014575,1.835746
19,(iron maiden),(metallica),0.058794,0.129048,0.031888,0.542373,4.202866,0.024301,1.903191
27,(tricky),(massive attack),0.031888,0.080219,0.017439,0.546875,6.817255,0.014881,2.029861


In [40]:
rules = association_rules(apr_itemsets, metric="confidence", min_threshold=0.5)
rules.sort_values(by = ['support'], ascending=False)

rules[ (rules['antecedents'].apply(lambda x: len(x)) >= 2) & (rules['confidence'] > 0.5) & (rules['lift'] > 1) & (rules['conviction'] > 2)].sort_values(by = ['support'], ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
37,"(pidżama porno, happysad)",(akurat),0.022920,0.048331,0.014449,0.630435,13.044151,0.013342,2.575105
38,"(pidżama porno, akurat)",(happysad),0.020927,0.053812,0.014449,0.690476,12.831349,0.013323,3.056916
39,"(happysad, akurat)",(pidżama porno),0.023916,0.050324,0.014449,0.604167,12.005569,0.013246,2.399182
60,"(koЯn, linkin park)",(system of a down),0.019930,0.106129,0.012955,0.650000,6.124648,0.010839,2.553918
51,"(massive attack, björk)",(portishead),0.018934,0.074738,0.011958,0.631579,8.450526,0.010543,2.511424
50,"(portishead, björk)",(massive attack),0.021923,0.080219,0.011958,0.545455,6.799548,0.010199,2.023518
62,"(led zeppelin, the doors)",(pink floyd),0.017937,0.099153,0.011460,0.638889,6.443467,0.009681,2.494653
35,"(happysad, myslovitz)",(akurat),0.020927,0.048331,0.011460,0.547619,11.330633,0.010448,2.103690
36,"(myslovitz, akurat)",(happysad),0.018435,0.053812,0.011460,0.621622,11.551802,0.010468,2.500641
43,"(coldplay, arctic monkeys)",(franz ferdinand),0.019930,0.056303,0.010962,0.550000,9.768584,0.009840,2.097105


кого предпочитают слушать user'ы в зависимости от пола 

кого предпочитают user'ы в зависимости от пола и места проживания (sex,country)->artist

для конкретного user (artist1,artist2)->artist3 (мб несколько цепочек) - 

In [158]:
rules[ 
       (rules['lift'] > 0.7) &
       (rules['support'] > 0.02) &
       (rules['antecedents']=={'placebo'})
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len


см. правила 

        ищем часто встречающиеся паттерны - artist1,artist2,artist3


In [157]:
rules[rules['antecedents']=={'ferry corsten'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3,(ferry corsten),(above & beyond),0.022422,0.021923,0.011958,0.533333,24.327273,0.011467,2.095879,1
7,(ferry corsten),(armin van buuren),0.022422,0.038864,0.015446,0.688889,17.725641,0.014575,3.089366,1
11,(ferry corsten),(atb),0.022422,0.048829,0.011958,0.533333,10.922449,0.010863,2.038223,1
17,(ferry corsten),(paul van dyk),0.022422,0.030892,0.011958,0.533333,17.264516,0.011266,2.076660,1


In [152]:
rules[rules['consequents']=={'muse'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
52,"(coldplay, franz ferdinand)",(muse),0.019432,0.101644,0.010463,0.538462,5.297511,0.008488,1.946437,2
54,"(coldplay, nirvana)",(muse),0.019930,0.101644,0.010463,0.525000,5.165074,0.008438,1.891275,2
67,"(placebo, nirvana)",(muse),0.022920,0.101644,0.012456,0.543478,5.346867,0.010127,1.967827,2
69,"(red hot chili peppers, placebo)",(muse),0.020927,0.101644,0.011460,0.547619,5.387605,0.009333,1.985839,2


In [153]:
rules[rules['consequents']=={'placebo'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
68,"(muse, nirvana)",(placebo),0.024415,0.102641,0.012456,0.510204,4.970775,0.009950,1.832108,2
70,"(the killers, muse)",(placebo),0.022422,0.102641,0.011958,0.533333,5.196117,0.009657,1.922913,2
